In [1]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

%matplotlib widget
# %matplotlib inline

import ipywidgets as widgets

%config InlineBackend.figure_format = 'retina'

### cross correlation: correlating 2 different time series

In [2]:
arr1 = np.array([0,1,2,1,0])
arr2 = np.array([0,1,0,2])
sig.correlate(arr1,arr2)
# notice Which is the reference and whice is the target, what led to this result?

array([0, 2, 4, 3, 2, 1, 0, 0])

### let us generate 2 neurons that have a base firing rate of 10 sp/s and react to a stimulus with a spike after a noisy delay:

In [3]:
# set parameters for the spike trains:
samp = 1000
rate = 10 / samp
duration = 360
total = samp*duration
dt = 1/samp

def interactive_cross_correlation(delay1, delay2, std, pfire1, pfire2, stim_freq):
#   create poisson spike trains: 
    spk_train1 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
    spk_train2 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
#   add a spike after every stimulation with noisy delay 
    for i in np.arange(100,total,int(samp/stim_freq)):
        if np.random.uniform()<pfire1:
            spk_train1[i+delay1+np.rint(std*np.random.randn(1)).astype(np.int16)] = 1
        if np.random.uniform()<pfire2:
            spk_train2[i+delay2+np.rint(std*np.random.randn(1)).astype(np.int16)] = 1
#   calculate autocorrelation:
    autocorr = sig.correlate(spk_train1, spk_train2, mode='full', method='auto')/sum(spk_train2)/dt
#   smooth autocorrelation:  
    window = sig.gaussian(15, 6)
    smoothed_autocorr = np.convolve(autocorr, window, 'same')/sum(window)
#   plot
    fig, ax = plt.subplots(figsize=(10,3))
    ax.plot(np.arange(-0.5,0.5,dt),smoothed_autocorr[(total-500):(total+500)])
    ax.set_xlabel('Offset(sec)')
    ax.set_ylabel('rate(sp/s)')
    ax.set_title('cross correlation interactive plot')
    
    fig.tight_layout()
    
_ = widgets.interact(interactive_cross_correlation, delay1 = (5,60,5), delay2 = (5,60,5), std=(1,20,1), pfire1=(0.1,1,0.1), pfire2=(0.1,1,0.1), stim_freq=[5,10,15])

interactive(children=(IntSlider(value=30, description='delay1', max=60, min=5, step=5), IntSlider(value=30, de…

#### Observe the cross correlation above, change the parameters and try to answer the following questions:
#### 1) Which neuron is the target and which is reference?
#### 2) What is the relation between these twho neurons? is the relation dependent on some of the parameters?
#### 3) If we calculated a shift predictor, how would it look like?
#### 4) What would we see if we observed a bigger window of offsets?
#### 5) How can we improve visualization of this autocorrelation?
#### 6) Observe the result of correlating the arrays of ones below, explain the result.

In [4]:
sig.correlate(np.ones(30),np.ones(30))

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 29., 28., 27., 26., 25., 24., 23., 22., 21.,
       20., 19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,
        7.,  6.,  5.,  4.,  3.,  2.,  1.])

#### Let us create a shift predictor and correct a cross correlation

#### in the following example, neuron 1 fires at a noisy delay after a stimulation while neuron 2 fires at a noisy delay after neuron 1 (at stimulation times only). We have calculated a shift predictor and used it to correct the cross correlation. 
#### Observe the following widget, try to answer the following:
#### 1) How is the shift predictor calculated? Does this cause problems?
#### 2) What can we learn from the shift predictor?
#### 3) What does the corrected cross correlation tell us?
#### 4) Describe another way to calculate shift predictor

In [5]:
# generate spike trains

samp = 1000
rate = 10 / samp
duration = 360
total = samp*duration
dt = 1/samp


def interactive_cross_correlation(delay1, delay2, std, pfire1, pfire2, stim_freq):

    spk_train1 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
    spk_train2 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)

    for i in np.arange(100,total,int(samp/stim_freq)):
        d1 = delay1+np.rint(std*np.random.randn(1)).astype(np.int16)
        if np.random.uniform()<pfire1:
            spk_train1[i+d1] = 1
        d2 = d1 + delay2 + np.rint(2*np.random.randn(1)).astype(np.int16)
        if np.random.uniform()<pfire2:
            spk_train2[i+d2] = 1

    autocorr = sig.correlate(spk_train1, spk_train2, mode='full', method='auto')/sum(spk_train2)/dt
    window = sig.gaussian(15, 4)
    smoothed_autocorr = np.convolve(autocorr, window, 'same')/sum(window)
    fig, ax = plt.subplots(nrows=3, figsize=(10,7), sharey=True)
    ax[0].plot(np.arange(-0.5,0.5,dt),smoothed_autocorr[(total-500):(total+500)])
    ax[0].set_xlabel('Time(sec)')
    ax[0].set_ylabel('rate(sp/s)')
    ax[0].set_title('cross correlation')
    
### calculate shift predictor

    shift_period = int(samp/stim_freq)*5 # amount of bins to shift by:
#   Shifting  
    shifted = np.roll(spk_train1, shift_period)
#   correlation with the shifted spike train to calculate the shift predictor:
    shift_predictor = sig.correlate(shifted, spk_train2, mode='full', method='auto')/sum(spk_train2)/dt
    window = sig.gaussian(15, 4)
    smoothed_shift_predictor = np.convolve(shift_predictor, window, 'same')/sum(window)
    ax[1].plot(np.arange(-0.5,0.5,dt),smoothed_shift_predictor[(total-500):(total+500)])
    ax[1].set_xlabel('Time(sec)')
    ax[1].set_ylabel('rate(sp/s)')
    ax[1].set_title('shift predictor')
#   using the shift predictor to correct the autocorrelation:
    corrected_autocorr = autocorr - shift_predictor
    window = sig.gaussian(8, 2)
    corrected_smoothed_autocorr = np.convolve(corrected_autocorr, window, 'same')/sum(window)
    ax[2].plot(np.arange(-0.5,0.5,dt),corrected_smoothed_autocorr[(total-500):(total+500)])
    ax[2].set_xlabel('Time(sec)')
    ax[2].set_ylabel('rate(sp/s)')
    ax[2].set_title('corrected cross correlation')
    plt.tight_layout()
    
    
_ = widgets.interact(interactive_cross_correlation, delay1 = (5,30,5), delay2 = (5,60,5), std=(1,20,1), pfire1=(0.1,1,0.1), pfire2=(0.1,1,0.1), stim_freq=[5,10,15])

interactive(children=(IntSlider(value=15, description='delay1', max=30, min=5, step=5), IntSlider(value=30, de…

### Peri Stimulus Time histogram - PSTH
#### Below you will find the psth of the neurons recorded in the same conditions as above 
#### Observe the following widget, try to answer the following:
#### 1) How is the PSTH calculated in the code?
#### 2) What are the differences between the upper and bottom plots? 
#### 3) what information can we derive from the psth?

In [12]:
def interactive_psth(delay1, delay2, std, pfire1, pfire2, stim_freq):

    spk_train1 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
    spk_train2 = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)

    trial_duration = int(samp/stim_freq)
    for i in np.arange(0,total,trial_duration):
        d1 = delay1+np.rint(std*np.random.randn(1)).astype(np.int16)
        if np.random.uniform()<pfire1:
            spk_train1[i+d1] = 1
        d2 = d1 + delay2 + np.rint(2*np.random.randn(1)).astype(np.int16)
        if np.random.uniform()<pfire2:
            spk_train2[i+d2] = 1

    psth1 = np.sum(spk_train1.reshape(-1, trial_duration), axis=0 )
    psth2 = np.sum(spk_train2.reshape(-1, trial_duration), axis=0 )
       
    window = sig.gaussian(6, 2)
    smoothed_psth1 = np.convolve(psth1, window, 'same')/sum(window)
    smoothed_psth2 = np.convolve(psth2, window, 'same')/sum(window)
    
    fig, ax = plt.subplots(nrows=2, figsize=(10,6), sharey=True)
    ax[0].bar(np.arange(0,trial_duration,1),psth1)
    ax[0].set_xlabel('Time(ms)')
    ax[0].set_ylabel('count(spikes)')
    ax[0].set_title('PSTH neuron 1')

    ax[1].plot(np.arange(0,trial_duration,1),smoothed_psth2)
    ax[1].set_xlabel('Time(ms)')
    ax[1].set_ylabel('count(spikes)')
    ax[1].set_title('PSTH neuron 2')
    
    fig.tight_layout()
    
    
_ = widgets.interact(interactive_psth, delay1 = (5,50,5), delay2 = (5,50,5), std=(1,20,1), pfire1=(0.1,1,0.1), pfire2=(0.1,1,0.1), stim_freq=[5,10])

interactive(children=(IntSlider(value=25, description='delay1', max=50, min=5, step=5), IntSlider(value=25, de…